In [ ]:
import numpy as np # numerical computation
import pandas as pd # # data processing/manipulation
import matplotlib.pyplot as plt # basic data visualization
# import seaborn as sns # nicer data visualization
import time

# stopwords, tokenizer, stemmer
import nltk  
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet



import re # regular expressions


import gensim #new nltk library for language processing
from gensim.parsing.preprocessing import remove_stopwords # we also use gensim for stopwords removal

from textblob import TextBlob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path

out_folder = '/content/drive/MyDrive/tweetdata' # folder to save intermediary data to
# Path(out_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
biden_csv_path = out_folder + '/biden_usa_unique_df.csv'
trump_csv_path = out_folder + '/trump_usa_unique_df.csv'

In [ ]:
biden_usa_unique_df = pd.read_csv(biden_csv_path, lineterminator='\n')
trump_usa_unique_df = pd.read_csv(trump_csv_path, lineterminator='\n')

In [ ]:
clean_tweets_path = out_folder + '/clean_tweets_df.csv'
tweets_df = pd.read_csv(clean_tweets_path, lineterminator='\n')

In [ ]:
tweets_df.drop(columns=['blob_polarity', 'blob_subjectivity', 'blob_sentiment'], axis=1, inplace=True)

In [ ]:
tweets_df.head(10)[['tweet', 'clean_tweet_nltk']]

,tweet,clean_tweet_nltk
0,Twitter is doing everything they can to help D...,twitter help democrat win election make sure s...
1,Come on @ABC PLEASE DO THE RIGHT THING. Move t...,come right thing biden town hall bet requireme...
2,#realDonaldTrump addresses #JoeBiden and #Hunt...,realdonaldtrump address joebiden hunterbiden c...
3,"Hunter #Biden introduced his father, then-Vice...",hunter biden introduce father president joe bi...
4,This is from the same night I met the cast of ...,night met cast tww wear shirt vote voteblue jo...
5,@realDonaldTrump #TrumpIsALaughingStock @realD...,trumpisalaughingstock iowa cult rally compare ...
6,Laptop computer abandoned at Delaware repair s...,laptop abandon delaware repair shop contains e...
7,Vice President #Biden is 77 years young.\nI am...,vice president biden year young willing bet lo...
8,So the #FBI received the LAPTOP with the EMAIL...,fbi receive laptop email instead act behalf am...
9,And Trump claims Biden has dementia? \n#Donald...,trump claim biden dementia donaldout trumprall...


https://www.kaggle.com/code/amineabouothmane/biden-vs-trump-sentiment-analysis-of-tweets

https://www.kaggle.com/code/nkitgupta/aspect-based-sentiment-analysis

We are using this:
https://medium.com/nlplanet/quick-intro-to-aspect-based-sentiment-analysis-c8888a09eda7

https://medium.com/analytics-vidhya/aspect-based-sentiment-analysis-a-practical-approach-8f51029bbc4a



### Using a public pre-trained model

First, we install the transformers library along with the SentencePiece tokenizer (which is needed by some models of the library, such as DeBERTa).

In [ ]:
pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 55.3 MB/s 
     |████████████████████████████████| 7.6 MB 45.3 MB/s 
     |████████████████████████████████| 1.3 MB 41.8 MB/s 


Then we import the necessary libraries and load two different models:

* The absa_model and absa_tokenizer to test the deberta-v3-base-abas-v1.1  re-trained ABSA model.
* The sentiment_model to test a standard sentiment model (optional). We’ll try the twitter-xlm-roberta-base-sentiment model, trained on ~198M tweets and finetuned for sentiment analysis.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

Downloading:   0%|          | 0.00/372 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


...

Downloading:   0%|          | 0.00/738M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
def compute_absa_scores(sentence, apsect):
  inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
  outputs = absa_model(**inputs)
  probs = F.softmax(outputs.logits, dim=1)
  probs = probs.detach().numpy()[0]
  return probs


In [ ]:
dict(zip(["negative", "neutral", "positive"], compute_absa_scores(sentence, aspect)))

{'negative': 0.99461293, 'neutral': 0.0023699857, 'positive': 0.0030170798}

https://intellica-ai.medium.com/aspect-based-sentiment-analysis-everything-you-wanted-to-know-1be41572e238

In [ ]:
fil = tweets_df['clean_tweet_nltk'].str.contains("covid")
fil[fil.isna()] = False

In [ ]:
covid_tweets = tweets_df[fil]
covid_tweets.head()['clean_tweet_nltk']

In [ ]:
covid_tweets.head()['clean_tweet_nltk']

150    use honor refuse covid test enter studio hell ...
426    shock remake unoriginality hollywood day chris...
622    joebiden kamalaharris morningjoe maddow nbc am...
777    diary radio junkie day wake news amyconeybarre...
787    diary radio junkie day wake news amyconeybarre...
Name: clean_tweet_nltk, dtype: object

In [ ]:
%%time
result = covid_tweets['clean_tweet_nltk'].apply(lambda row: compute_absa_scores(row, 'covid'))

CPU times: user 19min 53s, sys: 38.1 s, total: 20min 31s
Wall time: 20min 28s


In [ ]:
print(result)

150         [0.9247004, 0.044610262, 0.030689381]
426         [0.9152765, 0.070762165, 0.013961305]
622       [0.0055859108, 0.98925704, 0.005157038]
777         [0.010428367, 0.9791763, 0.010395332]
787          [0.00889568, 0.98186076, 0.00924359]
                           ...                   
133925     [0.9885039, 0.009941336, 0.0015548241]
133930     [0.9871034, 0.011554041, 0.0013425314]
133931     [0.9885039, 0.009941336, 0.0015548241]
133932     [0.9885039, 0.009941336, 0.0015548241]
134010       [0.9280375, 0.052911982, 0.01905052]
Name: clean_tweet_nltk, Length: 2802, dtype: object


In [ ]:
covid_absa_df = pd.DataFrame(result.tolist(), columns = ["covid_negative", "covid_neutral", "covid_positive"])

In [ ]:
covid_absa_df.head()

,covid_negative,covid_neutral,covid_positive
0,0.924700,0.044610,0.030689
1,0.915277,0.070762,0.013961
2,0.005586,0.989257,0.005157
3,0.010428,0.979176,0.010395
4,0.008896,0.981861,0.009244


In [ ]:
# Drop the index to concatenate two dataframes
covid_tweets_df = pd.concat([covid_tweets.reset_index(drop=True), covid_absa_df.reset_index(drop=True)], axis=1)
covid_tweets_df


,created_at,tweet_id,tweet,likes,retweet_count,user_id,user_followers_count,user_location,lat,long,...,state,state_code,lang,ds,tokenized_tweet_nltk,clean_tweet_nltk,"(covid_negative, covid_neutral, covid_positive)",covid_negative,covid_neutral,covid_positive
0,2020-10-15 01:21:39,1.316550e+18,@MarkHarrisNYC They'll use the honor system wh...,1.0,0.0,7.623333e+17,32.0,New york,40.712728,-74.006015,...,New York,NY,en,biden,"['use', 'honor', 'refuse', 'covid', 'test', 'e...",use honor refuse covid test enter studio hell ...,"[0.9247004, 0.044610262, 0.030689381]",0.924700,0.044610,0.030689
1,2020-10-15 04:59:59,1.316605e+18,What a shock!! Another remake!!! I cant take a...,0.0,0.0,2.163160e+07,407.0,Brooklyn,40.650102,-73.949583,...,New York,NY,en,biden,"['shock', 'remake', 'unoriginality', 'hollywoo...",shock remake unoriginality hollywood day chris...,"[0.9152765, 0.070762165, 0.013961305]",0.915277,0.070762,0.013961
2,2020-10-15 12:16:26,1.316715e+18,#JoeBiden #KamalaHarris #MorningJoe #Maddow #N...,1.0,0.0,2.636866e+08,982.0,New York City,40.712728,-74.006015,...,New York,NY,en,biden,"['joebiden', 'kamalaharris', 'morningjoe', 'ma...",joebiden kamalaharris morningjoe maddow nbc am...,"[0.0055859108, 0.98925704, 0.005157038]",0.005586,0.989257,0.005157
3,2020-10-15 14:15:06,1.316744e+18,Diary of a Radio Junkie: 1790 Days of Waking U...,1.0,0.0,3.877750e+08,523.0,"New York, USA",40.712728,-74.006015,...,New York,NY,en,biden,"['diary', 'radio', 'junkie', 'day', 'wake', 'n...",diary radio junkie day wake news amyconeybarre...,"[0.010428367, 0.9791763, 0.010395332]",0.010428,0.979176,0.010395
4,2020-10-15 14:23:54,1.316747e+18,@publicbill Diary of a Radio Junkie: 1790 Days...,0.0,0.0,3.877750e+08,523.0,"New York, USA",40.712728,-74.006015,...,New York,NY,en,biden,"['diary', 'radio', 'junkie', 'day', 'wake', 'n...",diary radio junkie day wake news amyconeybarre...,"[0.00889568, 0.98186076, 0.00924359]",0.008896,0.981861,0.009244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,2020-11-08 23:05:03,1.325575e+18,@realDonaldTrump @marklevinshow If trump put a...,0.0,0.0,1.902508e+08,89.0,"Washington, DC",38.894992,-77.036558,...,District of Columbia,DC,en,trump,"['trump', 'effort', 'fight', 'covid', 'fight',...",trump effort fight covid fight election result...,"[0.9885039, 0.009941336, 0.0015548241]",0.988504,0.009941,0.001555
2798,2020-11-08 23:06:17,1.325575e+18,@realDonaldTrump @BreitbartNews Breitbart is f...,0.0,0.0,1.902508e+08,89.0,"Washington, DC",38.894992,-77.036558,...,District of Columbia,DC,en,trump,"['breitbart', 'fake', 'newz', 'trump', 'effort...",breitbart fake newz trump effort fight covid f...,"[0.9871034, 0.011554041, 0.0013425314]",0.987103,0.011554,0.001343
2799,2020-11-08 23:06:47,1.325576e+18,@WhiteHouse If trump put as much effort into f...,0.0,0.0,1.902508e+08,89.0,"Washington, DC",38.894992,-77.036558,...,District of Columbia,DC,en,trump,"['trump', 'effort', 'fight', 'covid', 'fight',...",trump effort fight covid fight election result...,"[0.9885039, 0.009941336, 0.0015548241]",0.988504,0.009941,0.001555
2800,2020-11-08 23:07:12,1.325576e+18,@WhiteHouseCEA @WhiteHouse If trump put as muc...,0.0,0.0,1.902508e+08,89.0,"Washington, DC",38.894992,-77.036558,...,District of Columbia,DC,en,trump,"['trump', 'effort', 'fight', 'covid', 'fight',...",trump effort fight covid fight election result...,"[0.9885039, 0.009941336, 0.0015548241]",0.988504,0.009941,0.001555


In [ ]:
covid_tweets_df.to_csv(out_folder + '/covid_tweets_absa.csv')

Covid aspect for biden and trump
Add more aspects
 - fraud
 - immigration

In [ ]:
fil1 = tweets_df['clean_tweet_nltk'].str.contains("fraud")
fil1[fil1.isna()] = False


In [ ]:
fraud_tweets = tweets_df[fil1]
fraud_tweets.head()['clean_tweet_nltk']

152     forgot draftdoggingcoward ineptofleadership fr...
413     possible scenario november trump tweet insists...
536     fraud alert voter post photo unsolicited ballo...
573     multiple hairtransplant procedure hair joebide...
1065    multiple hairtransplant procedure hair joebide...
Name: clean_tweet_nltk, dtype: object

In [ ]:
%%time
result1 = fraud_tweets['clean_tweet_nltk'].apply(lambda row: compute_absa_scores(row, 'fraud'))

CPU times: user 14min 45s, sys: 39.1 s, total: 15min 24s
Wall time: 15min 21s


In [ ]:
fraud_absa_df = pd.DataFrame(result1.tolist(), columns = ["fraud_negative", "fraud_neutral", "fraud_positive"])
fraud_tweets_update_df = pd.concat([fraud_tweets_df.reset_index(drop=True), fraud_absa_df.reset_index(drop=True)], axis=1)
fraud_tweets_update_df.to_csv(out_folder + '/fraud_tweets_absa.csv')

In [ ]:
fraud_tweets_update_df.head()

,created_at,tweet_id,tweet,likes,retweet_count,user_id,user_followers_count,user_location,lat,long,...,"(covid_negative, covid_neutral, covid_positive)",covid_negative,covid_neutral,covid_positive,fraud_negative,fraud_neutral,fraud_positive,fraud_negative,fraud_neutral,fraud_positive
0,2020-10-15 01:21:39,1.316550e+18,@MarkHarrisNYC They'll use the honor system wh...,1.0,0.0,7.623333e+17,32.0,New york,40.712728,-74.006015,...,"[0.9247004, 0.044610262, 0.030689381]",0.924700,0.044610,0.030689,0.917480,0.065489,0.017031,0.917480,0.065489,0.017031
1,2020-10-15 04:59:59,1.316605e+18,What a shock!! Another remake!!! I cant take a...,0.0,0.0,2.163160e+07,407.0,Brooklyn,40.650102,-73.949583,...,"[0.9152765, 0.070762165, 0.013961305]",0.915277,0.070762,0.013961,0.182253,0.808786,0.008961,0.182253,0.808786,0.008961
2,2020-10-15 12:16:26,1.316715e+18,#JoeBiden #KamalaHarris #MorningJoe #Maddow #N...,1.0,0.0,2.636866e+08,982.0,New York City,40.712728,-74.006015,...,"[0.0055859108, 0.98925704, 0.005157038]",0.005586,0.989257,0.005157,0.352139,0.629959,0.017903,0.352139,0.629959,0.017903
3,2020-10-15 14:15:06,1.316744e+18,Diary of a Radio Junkie: 1790 Days of Waking U...,1.0,0.0,3.877750e+08,523.0,"New York, USA",40.712728,-74.006015,...,"[0.010428367, 0.9791763, 0.010395332]",0.010428,0.979176,0.010395,0.251474,0.622004,0.126522,0.251474,0.622004,0.126522
4,2020-10-15 14:23:54,1.316747e+18,@publicbill Diary of a Radio Junkie: 1790 Days...,0.0,0.0,3.877750e+08,523.0,"New York, USA",40.712728,-74.006015,...,"[0.00889568, 0.98186076, 0.00924359]",0.008896,0.981861,0.009244,0.520627,0.390251,0.089122,0.520627,0.390251,0.089122
